In [1]:
# import pandas as pd
# import numpy as np
# import os
# filePath = r"UCI HAR Dataset/train/Inertial Signals/"
# mainDir = r"UCI HAR Dataset/"
# trainGroup = "train"
# testGroup = "test"
# fileList = os.listdir(filePath)
# fileList

In [2]:
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils.np_utils import to_categorical
from keras import metrics
from matplotlib import pyplot
import tensorflow as tf

In [13]:
# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
	# load all train
	trainX, trainy = load_dataset_group('train', prefix + 'HARDataset/')
	print(trainX.shape, trainy.shape)
	# load all test
	testX, testy = load_dataset_group('test', prefix + 'HARDataset/')
	print(testX.shape, testy.shape)
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)
	print(trainX.shape, trainy.shape, testX.shape, testy.shape)
	return trainX, trainy, testX, testy

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
	verbose, epochs, batch_size = 0, 8, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	model = Sequential()
	model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
	model.add(Dropout(0.5))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, validation_data=(testX, testy))
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size)
	return accuracy



    
trainX, trainy, testX, testy = load_dataset()
score = evaluate_model(trainX, trainy, testX, testy)

# run the experiment
# run_experiment()

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
Epoch 1/8
115/115 [==============================] - 12s 78ms/step - loss: 1.4396 - accuracy: 0.3822 - val_loss: 1.1202 - val_accuracy: 0.5151
Epoch 2/8
115/115 [==============================] - 8s 68ms/step - loss: 1.1168 - accuracy: 0.5150 - val_loss: 0.8288 - val_accuracy: 0.6749
Epoch 3/8
115/115 [==============================] - 8s 70ms/step - loss: 0.6880 - accuracy: 0.7094 - val_loss: 0.7571 - val_accuracy: 0.7136
Epoch 4/8
115/115 [==============================] - 8s 70ms/step - loss: 0.5326 - accuracy: 0.7901 - val_loss: 0.8956 - val_accuracy: 0.6620
Epoch 5/8
115/115 [==============================] - 8s 69ms/step - loss: 0.5905 - accuracy: 0.7768 - val_loss: 0.5885 - val_accuracy: 0.8032
Epoch 6/8
115/115 [==============================] - 8s 68ms/step - loss: 0.3495 - accuracy: 0.8727 - val_loss: 0.5129 - val_accuracy: 0.8507
Epoch 7/8
115/115 [============================

In [ ]:
score